In [1]:
import os
import threading
import time

import CFV_Extractor as CE

In [2]:
ifWords = ["if","else if"]
keyWords = ["if","while","for","switch","else","else if"]

In [3]:
def pre_process_filter(nodeSet):
    filter_list = ["case","?:","break","default","continue"]
    keys = ["switch"]
    for i in range(0,len(nodeSet)):
        index = 0
        while index<len(nodeSet[i]["cfv"]):
            if nodeSet[i]["cfv"][index] in filter_list:
                del nodeSet[i]["cfv"][index]
            elif nodeSet[i]["cfv"][index] in keys:
                nodeSet[i]["cfv"][index] = "if"
            else:
                index+=1
        
    return nodeSet

def getGpickleList(path):
    name_list = []
    for filename in os.listdir(path):
        if filename.endswith(".gpickle"):
            name_list.append(os.path.join(path,filename))
    return name_list

def scanIFinEnvironment(nodeSet):
    node_if_numbers = [0] *  len(nodeSet)
    for index in range(0,len(nodeSet)):
        cfv = nodeSet[index]["cfv"]
        for i in range(0,len(cfv)):
            if cfv[i] == "if" or cfv[i] == "else if":
                node_if_numbers[index] += 1
    return node_if_numbers

def scanInputDeepth(nodeSet):
    node_if_numbers = scanIFinEnvironment(nodeSet)
    high = 0
    #for i in range(0,len(node_if_numbers)):
    #    if node_if_numbers[i]>high:
    #        high = node_if_numbers[i]
    high = sum(node_if_numbers)
    return high

def countWeight(cfv):
        weight = 0
        cur_weight = 0
        record = []
        for item in cfv:
            if item in keyWords:
                record.append(item)
                if len([x for x in record if x in ifWords]) > weight:
                    weight = len([x for x in record if x in ifWords])
            else:
                if item == "}":
                    if len(record) > 0:
                        record.pop()          
        return weight
    
def initWeight(cfvSet):
        weight = []
        for item in cfvSet:
            weight.append(countWeight(item["cfv"]))
        return weight

def scanVulDeepth(nodeSet,extractor):
    cfvWeight = initWeight(nodeSet)

    allPath = [[[0]]]

    for target in range(1,len(cfvWeight)):
        allPath.append(list(extractor.getAllPath(0,target)))

    #print(allPath)
    #print(cfvWeight)

    high = 0
    for i in range(0,len(cfvWeight)):
        for path in allPath[i]:
            score = sum([cfvWeight[x] for x in path])
            #print(path)
            #print([cfvWeight[x] for x in path])
            #print(score)
            if score >high:
                high = score
                
    return high

In [4]:
path = r"..\gpickle\openlaunchd_1.gpickle"


extractor = CE.CFV_Extractor(path)
nodeSet = extractor.getAllCFV()
nodeSet = pre_process_filter(nodeSet)

input_deepth = scanInputDeepth(nodeSet)
vul_deepth = scanVulDeepth(nodeSet,extractor)

print("input_deepth = "+str(input_deepth))
print("vul_deepth = "+str(vul_deepth))

input_deepth = 4
vul_deepth = 1


In [5]:
print(nodeSet)

[{'name': 'main', 'full_definition': 'int main void', 'id': 'wait4path_8c_1a0ddf1224851353fc92bfbff6f499fa97', 'bodyfile': '/home/hp5_xgz/github_dl/_OPstart/openlaunchd/wait4path/wait4path.c', 'bodystart': '31', 'bodyend': '61', 'refids': ['No_refs'], 'refedids': [], 'depth_binary_tree': 3, 'number_blocks': 6, 'number_code_lines': 27, 'cfv': ['{', ';', ';', ';', 'if', '{', ';', ';', '}', ';', 'if', '{', ';', ';', '}', 'if', '{', ';', '}', 'for', '{', ';', 'if', '{', ';', '}', '}', ';', '}'], 'refDef': []}]


In [6]:
for i in range(0,len(nodeSet)):
    print(nodeSet[i]["full_definition"])

int main void
